In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Retinoblastoma"
cohort = "GSE63529"

# Input paths
in_trait_dir = "../../input/GEO/Retinoblastoma"
in_cohort_dir = "../../input/GEO/Retinoblastoma/GSE63529"

# Output paths
out_data_file = "../../output/preprocess/Retinoblastoma/GSE63529.csv"
out_gene_data_file = "../../output/preprocess/Retinoblastoma/gene_data/GSE63529.csv"
out_clinical_data_file = "../../output/preprocess/Retinoblastoma/clinical_data/GSE63529.csv"
json_path = "../../output/preprocess/Retinoblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Comparison of parental and CDKi-resistant ovarian cancer cell lines"
!Series_summary	"High-grade serous ovarian cancers (HGSOC) are genomically complex, heterogeneous cancers with a high mortality rate, due to acquired chemoresistance and lack of targeted therapy options. Cyclin-dependent kinase inhibitors (CDKi) target the retinoblastoma (RB) signaling network, and have been successfully incorporated into treatment regimens for breast and other cancers. Here, we have compared mechanisms of response and resistance to three CDKi that target either CDK4/6 or CDK2 and abrogate E2F target gene expression. We identify CCNE1 gain and RB1 loss as mechanisms of resistance to CDK4/6 inhibition, whereas receptor tyrosine kinase (RTK) and RAS signaling is associated with CDK2 inhibitor resistance. Mechanistically, we show that ETS factors are mediators of RTK/RAS signaling that cooperate with E2F in cell cycle progression. Consequently, CDK2 inhibition sensi

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from cell lines
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Retinoblastoma): Not directly available in sample characteristics
# However, the dataset is about Retinoblastoma (RB) signaling network in ovarian cancer
# This is not patient-level data for Retinoblastoma, but cell lines
trait_row = None  # Not available at sample level 

# For age: Not available in this dataset (cell line study)
age_row = None

# For gender: Not applicable for cell lines
gender_row = None

# 2.2 Data Type Conversion Functions

# For trait - not needed as data isn't available
def convert_trait(value):
    return None

# For age - not needed as data isn't available
def convert_age(value):
    return None

# For gender - not needed as data isn't available
def convert_gender(value):
    return None

# 3. Save Metadata
# Initial filtering to determine dataset usability
is_trait_available = (trait_row is not None)  # False in this case
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this step
# (No clinical feature extraction to perform)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers are Illumina probe IDs, not human gene symbols
# ILMN_ prefix is characteristic of Illumina BeadArray microarray probes
# These need to be mapped to human gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns contain probe IDs and gene symbols
# From the previous outputs, we can see:
# - In gene_data, the index has ILMN_ identifiers (Illumina probe IDs)
# - In gene_annotation, 'ID' column has the same ILMN_ format, and 'Symbol' column contains gene symbols

# 2. Get the gene mapping dataframe by extracting the relevant columns
probe_col = 'ID'           # Column containing probe IDs that match gene_data index
gene_col = 'Symbol'        # Column containing gene symbols

mapping_data = get_gene_mapping(gene_annotation, probe_col, gene_col)

# 3. Apply the gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Print the first few rows of the resulting gene expression data
print("Gene expression data after mapping (first 5 genes):")
print(gene_data.head(5))

# Print the shape of the gene expression data
print(f"\nGene expression data shape: {gene_data.shape}")


Gene expression data after mapping (first 5 genes):
       GSM1551822  GSM1551823  GSM1551824  GSM1551825  GSM1551826  GSM1551827  \
Gene                                                                            
A1BG     31.82586    49.63005    26.43715    30.91755   203.61640    44.67797   
A1CF     41.20912    33.54251    33.00000    33.00000    94.47501    33.00000   
A26C3    38.00962    33.00000    33.00000    40.55023    33.00000    33.00000   
A2BP1    44.00000    50.24579    44.00000    44.00000    51.90125    44.00000   
A2LD1   184.73540    26.63804   252.88430   181.11670   262.97600   169.51240   

       GSM1551828  GSM1551829  GSM1551830  GSM1551831  ...  GSM1551846  \
Gene                                                   ...               
A1BG     35.65852    30.68812    25.05865    25.17648  ...    22.00000   
A1CF     33.00000    33.00000    33.00000    33.00000  ...    33.72447   
A26C3    33.00000    33.00000    33.00000    33.00000  ...    37.76488   
A2BP1    4

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Since we determined in Step 2 that trait data (patient-level Retinoblastoma data) 
# is not available in this dataset, we can't proceed with linking clinical and genetic data
print("No clinical data available for patient-level Retinoblastoma in this dataset.")

# 3-5. For datasets without trait data, we use is_final=False in the validation
# This will record the dataset's metadata but not attempt final validation
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=False
)

# 6. Since the dataset doesn't have trait data, we don't save any linked data file
print(f"This dataset doesn't contain patient-level Retinoblastoma trait data and cannot be used for association studies.")

Normalized gene data shape: (20259, 34)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']


Normalized gene data saved to ../../output/preprocess/Retinoblastoma/gene_data/GSE63529.csv
No clinical data available for patient-level Retinoblastoma in this dataset.
This dataset doesn't contain patient-level Retinoblastoma trait data and cannot be used for association studies.
